<a href="https://colab.research.google.com/github/lephuocdat2000/DeepLearning-and-Application/blob/main/EngtoVie_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparing Data

###1, Import libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd



###2, Reading the data

In [2]:
!wget http://www.manythings.org/anki/vie-eng.zip -O viet-eng.zip
!unzip viet-eng.zip

--2021-06-26 07:44:15--  http://www.manythings.org/anki/vie-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.21.55.222, 2606:4700:3031::6815:37de, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.173.198|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 298429 (291K) [application/zip]
Saving to: ‘viet-eng.zip’

viet-eng.zip        100%[===================>] 291.43K  1.70MB/s    in 0.2s    

2021-06-26 07:44:15 (1.70 MB/s) - ‘viet-eng.zip’ saved [298429/298429]

Archive:  viet-eng.zip
  inflating: _about.txt              
  inflating: vie.txt                 


In [3]:
lines = pd.read_table( 'vie.txt' , names=[ 'eng' , 'vie' ] )
lines.reset_index( level=0 , inplace=True)
lines.rename( columns={ 'index' : 'eng' , 'eng' : 'vie' , 'vie' : 'c' } , inplace=True )
lines = lines.drop('c',1)

In [4]:
lines.head()

,eng,vie
0,Run!,Chạy!
1,Help!,Giúp tôi với!
2,Go on.,Tiếp tục đi.
3,Hello!,Chào bạn.
4,Hurry!,Nhanh lên nào!


### 3) Preparing input data for the Encoder ( `encoder_input_data` )

In [5]:
eng_lines = list()
for line in lines.eng:
    eng_lines.append( line ) 

tokenizer = preprocessing.text.Tokenizer()  
tokenizer.fit_on_texts( eng_lines ) 
tokenized_eng_lines = tokenizer.texts_to_sequences( eng_lines ) 

length_list = list()
for token_seq in tokenized_eng_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
padded_eng_lines = preprocessing.sequence.pad_sequences( tokenized_eng_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_eng_lines )

eng_word_dict = tokenizer.word_index
num_eng_tokens = len( eng_word_dict )+1

### 4) Preparing input data for the Decoder ( `decoder_input_data` )

In [6]:
!pip install underthesea

     |████████████████████████████████| 7.5MB 8.6MB/s 
     |████████████████████████████████| 51kB 9.6MB/s 
     |████████████████████████████████| 1.3MB 45.8MB/s 
     |████████████████████████████████| 747kB 44.8MB/s 
     |████████████████████████████████| 753.2MB 24kB/s 
     |████████████████████████████████| 245kB 59.5MB/s 
     |████████████████████████████████| 2.9MB 43.2MB/s 
     |████████████████████████████████| 901kB 34.1MB/s 
     |████████████████████████████████| 1.1MB 43.0MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=b6b0478317adbb957e01e9d14dc9d6ca6ca94df870ec33ca2c92c1bc76ccc1ff
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.5.1 which is incompatible.
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.5.1 which is incompat

In [7]:
import re
strip_special_chars = re.compile("[^\w0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [8]:
from underthesea import word_tokenize
tokenized_vie_lines = []
dict_words = np.array([])
dict_numbers = np.array([]).astype('int64')
vie_lines = []
tokenized_vie_lines = []
vie_word_dict = dict()
for line in lines.vie:
    line = word_tokenize(line,format='text')  
    full_line = '<START> ' + line + ' <END>'
    full_line = cleanSentences(full_line)
    words = full_line.split()
    for word in words:
       where_word_arr = np.where(dict_words==word)[0]
       if len(where_word_arr)==0: 
           dict_words=np.append(dict_words,word)
           dict_numbers=np.append(dict_numbers,1)
       else: 
         dict_numbers[where_word_arr[0]]+=1
    vie_lines.append(full_line)

agr_sorted_dict_numbers = np.argsort(dict_numbers)[::-1]
for idx,value in enumerate(agr_sorted_dict_numbers):
    vie_word_dict.update({dict_words[value]:idx})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [9]:
for i,vie_line in enumerate(vie_lines):
    words = vie_line.split()
    words_arr = np.zeros(len(words),dtype=int)
    for i,word in enumerate(words):
       words_arr[i] = vie_word_dict[word]
    tokenized_vie_lines.append(words_arr)

length_list = list()
for token_seq in tokenized_vie_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array(length_list).max()



### 5) Preparing target data for the Decoder ( decoder_target_data ) 

In [10]:
decoder_target_data = list()
decoder_input_data = list()
for token_seq in tokenized_vie_lines:
    decoder_target_data.append( token_seq[ 1 : ] ) 
    decoder_input_data.append(token_seq[0:-1])

#decoder_input
padded_vie_lines = preprocessing.sequence.pad_sequences( decoder_input_data , maxlen=max_output_length , padding='post' )
decoder_input_data = np.array([padded_vie_lines])
decoder_input_data = decoder_input_data.reshape(decoder_input_data.shape[1],decoder_input_data.shape[2])
num_vie_tokens = len(dict_words )+1

padded_vie_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_vie_lines = utils.to_categorical( padded_vie_lines , num_vie_tokens)
decoder_target_data = np.array( onehot_vie_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

Decoder target data shape -> (7547, 36, 3542)


#Defining and Training the models

###1, Defining the Encoder - Decoder model

In [11]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_eng_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 128 , return_state=True  )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_vie_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 128 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_vie_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    950272      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    906752      input_2[0][0]                    
______________________________________________________________________________________________

###Training model

In [12]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=250, epochs=50 ) 
model.save( 'model.h5' ) 

Epoch 1/50
31/31 [==============================] - 18s 191ms/step - loss: 1.2851
Epoch 2/50
31/31 [==============================] - 6s 190ms/step - loss: 1.1379
Epoch 3/50
31/31 [==============================] - 6s 188ms/step - loss: 1.1138
Epoch 4/50
31/31 [==============================] - 6s 191ms/step - loss: 1.0966
Epoch 5/50
31/31 [==============================] - 6s 192ms/step - loss: 1.0802
Epoch 6/50
31/31 [==============================] - 6s 193ms/step - loss: 1.0634
Epoch 7/50
31/31 [==============================] - 6s 188ms/step - loss: 1.0459
Epoch 8/50
31/31 [==============================] - 6s 190ms/step - loss: 1.0280
Epoch 9/50
31/31 [==============================] - 6s 192ms/step - loss: 1.0099
Epoch 10/50
31/31 [==============================] - 6s 193ms/step - loss: 0.9917
Epoch 11/50
31/31 [==============================] - 6s 189ms/step - loss: 0.9728
Epoch 12/50
31/31 [==============================] - 6s 186ms/step - loss: 0.9541
Epoch 13/50
31/31 [=====

#Save weights

In [13]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [14]:
!cp model.h5 /content/drive/MyDrive

#Load weights

In [ ]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/model_engtovie.h5')

In [17]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 128 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 128 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( eng_word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')

In [18]:
enc_model , dec_model = make_inference_models()

In [ ]:
for epoch in range( encoder_input_data.shape[0] ):
    tokenized_input = str_to_tokens( input( 'Enter eng sentence : ' ) )
    states_values = enc_model.predict(tokenized_input)
    #states_values = enc_model.predict( encoder_input_data[ epoch ] )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = vie_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in vie_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )
